# Auto Annotations


In [1]:
cd ..

/Users/yanndubois/Desktop/GitHub/alpaca_farm


## Setting up

All of our annotators currently use OpenAI API. So the first step is to setup your OpenAI key (and potentially your organization). This can be done by either running setting your environment variable like 
- `export OPENAI_API_KEY="sk..."` 

or by setting the following python variables:

In [2]:
import os
if 'OPENAI_API_KEY' not in os.environ:
    decoding_kwargs = dict(
        openai_api_key = None, #"sk-...",
        openai_organization_ids = None, # ["org-...","org-..."] you can set multiple orgs to avoid rate limits
    )
    assert decoding_kwargs["openai_api_key"] is not None, "OPENAI_API_KEY not found you should set it in environment or above"
else:
    decoding_kwargs = {}

The key object that you need for automatic annotations (both for training or for eval) is our `PairwiseAutoAnnotator` which inherits from [AlapcaEval's `PairwiseAnnotator`](https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/annotators/pairwise_evaluator.py#L20). By default the annotator is the pool from Alpaca Farm, for simplicity let's use a single annotator `test` for now.

For an overview annotators including how to extend them (different model such as ChatGPT or GPT4, prompts, decoding parameters), refer to the [Configuring annotators](#Configuring-annotators) section of this notebook. For more details refer to [AlpacaEval](https://github.com/tatsu-lab/alpaca_eval/tree/main).

In [3]:
from alpaca_farm.utils import jload
from alpaca_farm.auto_annotations import PairwiseAutoAnnotator

annotator = PairwiseAutoAnnotator(annotators_config="test", **decoding_kwargs)

INFO:root:Creating the annotator from `test`.
INFO:root:Saving annotations to `/Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json`.
INFO:root:Loading all annotations from /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.


## Annotating paired outputs

Now let's annotate some pairwise preference. All we need is some data. The annotator takes in either list of dictionaries, or pandas dataframes. The keys of dictionaries (or columns in dataframe) need to always contain `instruction` and `input` which defines the instruction. The annotator also needs a pair of outputs to compare, if you have a sequence of such pairs under the keys `output_1` and `output_2` then you can directly use `annotate_pairs`.

In [4]:
outputs_pairs = jload("examples/data/outputs_pairs.json")[:6]
print("Example of paired output:\n")
outputs_pairs[-1:]

Example of paired output:



[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': "Hey everyone! \n\nI'm hosting a dinner party this Friday night and I'd love for all of you to come over. We'll have a delicious spread of food and some great conversations. \n\nLet me know if you can make it - I'd love to see you all there!\n\nCheers,\n[Your Name]"}]

In [5]:
annotated = annotator.annotate_pairs(outputs_pairs)

INFO:root:Annotating 0 examples with davinci003_3
INFO:root:Saving all annotations to /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.
INFO:root:Loading all annotations from /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.


In [6]:
annotated[-1:]

[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': "Hey everyone! \n\nI'm hosting a dinner party this Friday night and I'd love for all of you to come over. We'll have a delicious spread of food and some great conversations. \n\nLet me know if you can make it - I'd love to see you all there!\n\nCheers,\n[Your Name]",
  'annotator': 'davinci003_3',
  'preference': 1}]

Here we see that the annotations adds two keys:
- `'preference'`: the index of the preferred output, here `preference=2` so `output_1` is prefered. In the case where both outputs are the same we give `preference=0`
- `'annotator'`: the name of the simulated annotator as found in the `annotators_config`

`annotate_pairs` is the main function and should be used if you have paired outputs to annotate. In many usecases, however, you  will outputs in different formats. In the following we discuss two additional helper function `annotate_head2head` and `annotate_samples` which are paticularly well suited for the typical format during evaluation and training respectively. Both functions call `annotate_pairs` under the hood after a reformatting step.

## Evaluation through pairwise comparison
For evaluation we need two components:
- outputs from the model we want to compare
- outputs on the same examples from the baseline model

Often case both of those components will be in a different list of dictionary (one list for each model). In this case all dictionaries need to contain an `output` column. Let us load such data from our simulated RLHF model and text-davinci-003 baseline.

In [7]:
outputs_baseline = jload("examples/data/outputs_baseline.json")[:6]
print("Example of baseline output:\n")
outputs_baseline[-1:]

Example of baseline output:



[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]"}]

In [8]:
outputs_rlhf = jload("examples/data/outputs_rlhf.json")[:6]
print("Example of rlhf output:\n")
outputs_rlhf[-1:]

Example of rlhf output:



[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output': 'Dear Friends, \n\nI am writing to invite you all to a dinner on Friday evening. It is a casual affair, and I am looking forward to a fun evening catching up with you all. I am planning to make a selection of delicious dishes, ranging from appetizers to mains and desserts. There will be something for everyone to enjoy, and I am sure it will be a night to remember.\n\nThe dinner will be held at my place on Friday, April 17th at 7pm. If you are interested in joining me, please RSVP to this email by Thursday, April 16th. I am looking forward to seeing you all there! \n\nThank you, \n\n[Name]'}]

The annotator's function of interest when we have two sequences of outputs is `annotate_head2head`.

In [9]:
annotated = annotator.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_rlhf)

INFO:root:Annotating 0 examples with davinci003_3
INFO:root:Saving all annotations to /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.
INFO:root:Loading all annotations from /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.


In [10]:
annotated[:1]

[{'instruction': 'The sentence you are given might be too wordy, complicated, or unclear. Rewrite the sentence and make your writing clearer by keeping it concise. Whenever possible, break complex sentences into multiple sentences and eliminate unnecessary words.',
  'input': 'If you have any questions about my rate or if you find it necessary to increase or decrease the scope for this project, please let me know.',
  'output_1': 'If you have questions about my rate or need to modify the scope of this project, please let me know.',
  'output_2': 'I am available to answer any questions you may have about my rate or if you need to alter the scope of this project. Feel free to contact me if you have any queries or require any additional information.',
  'annotator': 'davinci003_3',
  'preference': 1}]

We see that the format of the output is the same as before. Here `preference` indicates that the simulator prefered the output `output_1`, which corresponds to the baseline model.

## Training
For training we typically have multiple outputs for each instruction which are sampled from the same (SFT) model as seen in the following data:

In [11]:
outputs_samples = jload("examples/data/multisamples_sft.json")[:1]
print("Example of different sampled outputs from SFT:\n")
outputs_samples

Example of different sampled outputs from SFT:



[{'instruction': 'Describe a time when you had to make a difficult decision.',
  'input': '',
  'output': ["I had to make a difficult decision a few years ago when I was offered a job that would require me to move to a different city. I had to weigh the pros and cons of this job offer, and consider what it would mean for me to leave my friends and family in order to take it. In the end, I decided to turn down the job offer since I wasn't ready to quit my current job, move to a different city, and leave my loved ones behind.",
   'I had to make a difficult decision last year when I was faced with a life-changing opportunity. I had to decide whether or not to leave my current job and move to a different city for the chance to further my career. After much consideration, I chose to take the risk and make the move. Thankfully, it paid off and I am now enjoying my new job and exploring the city.',
   "I had to make a difficult decision when I was offered a job in a city that was significant

In this case, you can use the annotator's `annotate_samples`, which first samples pairs of outputs that have the same instruction/input and then annotate those.

In [12]:
annotated = annotator.annotate_samples(outputs_samples)

INFO:root:Filtered unique instruction/input pairs: 4 -> 1
INFO:root:Annotating 0 examples with davinci003_3
INFO:root:Saving all annotations to /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.
INFO:root:Loading all annotations from /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.


In [13]:
annotated

[{'sample_id': 0,
  'instruction': 'Describe a time when you had to make a difficult decision.',
  'input': '',
  'output_1': "I had to make a difficult decision a few years ago when I was offered a job that would require me to move to a different city. I had to weigh the pros and cons of this job offer, and consider what it would mean for me to leave my friends and family in order to take it. In the end, I decided to turn down the job offer since I wasn't ready to quit my current job, move to a different city, and leave my loved ones behind.",
  'output_2': "I had to make a difficult decision when I was offered a job in a city that was significantly farther away from my friends and family. I weighed the pros and cons and consulted with people I trusted, but in the end I had to make the choice that was best for my future and my well-being. It was a difficult decision, but I'm glad I made the choice that allowed me to further my career and start my journey towards independence.",
  'ann

By default there will only be one pair per instruction,input. If you use `is_unique_instructions=False` then you will get as many pairs as outputs.    

## Going further

### Adding noise

During training we typically flip the the label with probability 0.25 to emulate the variability of human annotations. To do so you can either:
- initialize the annotator with `PairwiseAutoAnnotator(p_label_flip=0.25)`
- set the noise of an initialized annotator `annotator.set_noise(p_label_flip=0.25)` 
- give the noise to annotate_samples `annotate_samples(..., p_label_flip=0.25)` 

In [14]:
annotated = annotator.annotate_samples(outputs_samples, p_label_flip=0.25)

INFO:root:Filtered unique instruction/input pairs: 4 -> 1
INFO:root:Adding random noise to the labels p_label_flip=0.25.
INFO:root:Annotating 0 examples with davinci003_3
INFO:root:Saving all annotations to /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.
INFO:root:Loading all annotations from /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.


In [15]:
annotated

[{'sample_id': 0,
  'instruction': 'Describe a time when you had to make a difficult decision.',
  'input': '',
  'output_1': "I had to make a difficult decision a few years ago when I was offered a job that would require me to move to a different city. I had to weigh the pros and cons of this job offer, and consider what it would mean for me to leave my friends and family in order to take it. In the end, I decided to turn down the job offer since I wasn't ready to quit my current job, move to a different city, and leave my loved ones behind.",
  'output_2': "I had to make a difficult decision when I was offered a job in a city that was significantly farther away from my friends and family. I weighed the pros and cons and consulted with people I trusted, but in the end I had to make the choice that was best for my future and my well-being. It was a difficult decision, but I'm glad I made the choice that allowed me to further my career and start my journey towards independence.",
  'ann

### Cost & time efficiency: avoiding duplicate annotation
Often time you will find yourself reannotating examples that you already annotated. This is particularly true if you sample many times from the same model (eg to get pairwise preferences for training or to evaluate best-of-n) or if the instructions you are considering require short outputs => many models might give the same exact answer.

This means that you have to spend unecessary money and time. Thankfully `PairwiseAutoAnnotator` stores previous annotations and will reuse those. For example, let us reannotate the previous evaluation

In [16]:
annotated = annotator.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_rlhf)

INFO:root:Annotating 0 examples with davinci003_3
INFO:root:Saving all annotations to /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.
INFO:root:Loading all annotations from /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/test/annotations_seed0_configs.json.


In [17]:
annotated[-1:]

[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': 'Dear Friends, \n\nI am writing to invite you all to a dinner on Friday evening. It is a casual affair, and I am looking forward to a fun evening catching up with you all. I am planning to make a selection of delicious dishes, ranging from appetizers to mains and desserts. There will be something for everyone to enjoy, and I am sure it will be a night to remember.\n\nThe dinner will be held at my place on Friday, April 17th at 7pm. If you are interested in joining me, please RSVP to this email by Thursday, April 16th. I am looking forward to seeing you all there! \n\nThank you, \n\n[Name]',
  'annotator': '

We now get the annotations without actually having to reannotate any example.
By default, the annotations are saved on disk in the directory that contains the annotators configs. You can remove this caching by giving `caching_path=None` to `PairwiseAutoAnnotator`.


### Evaluating win rates 

Let's show how to get win rates from annotations using our actual pool of annotators

In [18]:
from alpaca_eval.metrics import pairwise_to_winrate

In [19]:
# all the following is selfinstruct eval
outputs_baseline = jload("examples/data/outputs_baseline.json")
outputs_rlhf = jload("examples/data/outputs_rlhf.json")
outputs_sft = jload("examples/data/outputs_sft.json")

In [20]:
annotator_pool = PairwiseAutoAnnotator()

INFO:root:Creating the annotator from `annotator_pool_v0`.
INFO:root:Saving annotations to `/Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/annotator_pool_v0/annotations_seed0_configs.json`.


In [21]:
annotated_sft = annotator_pool.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_sft)
pairwise_to_winrate(preferences=[a["preference"] for a in annotated_sft])

INFO:root:Annotating 18 examples with gpt4_1
INFO:root:Using `openai_completions` on 5 prompts using gpt-4-0314.
INFO:root:Kwargs to completion: {'max_tokens': 600, 'temperature': 1.0}
INFO:root:Kwargs to completion: {'n': 1, 'model': 'gpt-4-0314', 'is_chat': True, 'max_tokens': 600, 'temperature': 1.0}
prompt_batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:04<00:00, 12.94s/it]
INFO:root:Completed 5 examples in 64.8 seconds.
INFO:root:Annotating 20 examples with gpt4_2
INFO:root:Using `openai_completions` on 5 prompts using gpt-4-0314.
INFO:root:Kwargs to completion: {'max_tokens': 250, 'temperature': 1.0}
INFO:root:Kwargs to completion: {'n': 1, 'model': 'gpt-4-0314', 'is_chat': True, 'max_tokens': 250, 'temperature': 1.0}
prompt_batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.89s/it]
INFO:root:

{'win_rate': 30.555555555555557,
 'standard_error': 2.838766377491875,
 'n_wins': 72,
 'n_wins_base': 170,
 'n_draws': 10,
 'n_total': 252}

In [22]:
annotated_rlhf = annotator_pool.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_rlhf)
pairwise_to_winrate(preferences=[a["preference"] for a in annotated_rlhf])

INFO:root:Annotating 19 examples with gpt4_1
INFO:root:Using `openai_completions` on 6 prompts using gpt-4-0314.
INFO:root:Kwargs to completion: {'max_tokens': 600, 'temperature': 1.0}
INFO:root:Kwargs to completion: {'n': 1, 'model': 'gpt-4-0314', 'is_chat': True, 'max_tokens': 600, 'temperature': 1.0}
prompt_batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:21<00:00, 13.50s/it]
INFO:root:Completed 6 examples in 81.1 seconds.
INFO:root:Annotating 19 examples with gpt4_2
INFO:root:Using `openai_completions` on 4 prompts using gpt-4-0314.
INFO:root:Kwargs to completion: {'max_tokens': 250, 'temperature': 1.0}
INFO:root:Kwargs to completion: {'n': 1, 'model': 'gpt-4-0314', 'is_chat': True, 'max_tokens': 250, 'temperature': 1.0}
prompt_batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:26<00:00,  6.53s/it]
INFO:root:

{'win_rate': 47.42063492063492,
 'standard_error': 3.1454933555275866,
 'n_wins': 119,
 'n_wins_base': 132,
 'n_draws': 1,
 'n_total': 252}

In [23]:
import datasets
from alpaca_farm import constants
outputs_baseline = datasets.load_dataset("tatsu-lab/alpaca_farm",
                                             "alpaca_farm_evaluation",
                                             cache_dir=constants.DEFAULT_CACHE_DIR,
                                             )['eval']

/Users/yanndubois/.cache/huggingface/datasets/downloads/b8c17ea4d3dc2405e3c93ddaea295a6fe540c820557b668c6ac40cb66e1f606d


Generating eval split: 0 examples [00:00, ? examples/s]

Dataset alpaca_farm downloaded and prepared to /Users/yanndubois/.cache/huggingface/datasets/tatsu-lab___alpaca_farm/alpaca_farm_evaluation/1.0.0/79d38dc3f12abd62869e376303b68092e8385769e22f05166fe96a3dac29a57a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
annotator = PairwiseAutoAnnotator()

INFO:root:Creating the annotator from `annotator_pool_v0`.
INFO:root:Saving annotations to `/Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/annotator_pool_v0/annotations_seed0_configs.json`.
INFO:root:Loading all annotations from /Users/yanndubois/Desktop/GitHub/alpaca_farm/src/alpaca_farm/auto_annotations/annotators/annotator_pool_v0/annotations_seed0_configs.json.


In [25]:
outputs_2 = outputs_baseline.to_pandas()[:10]
outputs_2.output = outputs_2.output.str[:20]
outputs_1  = outputs_baseline.to_pandas()[:10]

In [26]:
annotated = annotator.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_2)

                    805, len(outputs_2)==10, and len(df_annotated)==10. 
                    This means that there are missing examples or duplicates. We are taking a SQL inner join.
                    
INFO:root:Annotating 0 examples with gpt4_1
INFO:root:Annotating 2 examples with gpt4_2
INFO:root:Using `openai_completions` on 1 prompts using gpt-4-0314.
INFO:root:Kwargs to completion: {'max_tokens': 250, 'temperature': 1.0}
INFO:root:Kwargs to completion: {'n': 1, 'model': 'gpt-4-0314', 'is_chat': True, 'max_tokens': 250, 'temperature': 1.0}
prompt_batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.74s/it]
INFO:root:Completed 1 examples in 17.8 seconds.
INFO:root:Annotating 0 examples with gpt4_3
INFO:root:Annotating 0 examples with gpt4_4
INFO:root:Annotating 0 examples with gpt4_5
INFO:root:Annotating 0 examples with chatgpt_1
INFO:root:Annotating 2 examples with chatgpt_2
INFO:roo

### Configuring annotators

The most important argument to `PairwiseAutoAnnotator` is `annotators_config` which defines the pool of annotators (the API provider, the model, the prompt, and the decoding parameters) . We provide the following options out of the box:
- `annotators_config="annotators/annotator_pool_v0/configs.yaml"`: ApacaFarm's default annotator pool
- `annotators_config="annotators/greedy_gpt4/configs.yaml"`: Greedy GPT4 annotator
- `annotators_config="annotators/test/configs.yaml"`: a faster text-davinci-003 annotator useful for testing

Here's the desciption of `annotators_config` from the docstring:
```
A dictionary or path to a yaml file containing the configuration for the pool of annotators. The keys in the first dictionary should be the annotator's name, and the value should be a dictionary of the annotator's configuration which should have the following keys:

- prompt_templates (dict): a dictionary of prompt templates or path to the prompts. The keys should be "without_inputs" and "with_inputs". Each template should contain placeholders for keys in the example dictionary, typically {instruction} and {output_1} {output_2}.
- fn_decoder (str): function in `alpaca_farm.auto_annotations.pairwise_annotators.decoders.py` for completions.
- decoder_kwargs (dict): kwargs for fn_decode. E.g. model_name, max_tokens, temperature, tokens_to_avoid, tokens_to_favor
- outputs_to_match (dict): Kwargs for fn_completion_parser. With the default regex parser it needs `outputs_to_match` which is a dictionary of outputs to match from the completions. The values should be a regex pattern that should be matched, the keys should be the corresponding preference value. For example {1: 'Output \(a\)'} will match the output "Output (a)" and set the preference to 1.
- other kwargs to `SinglePairwiseAutoAnnotator` such as batch_size
```

And here is config `"annotators/test/configs.yaml"` of the annotator we used above. 

In [27]:
!cat src/alpaca_farm/auto_annotations/annotators/test/configs.yaml

davinci003_3 : # text-davinci-003_v1_b5-pairwise_temp=1.0
  prompt_template:
    with_inputs: "annotator_pool_v0/text_b5_with_inputs.txt"
    without_inputs: "annotator_pool_v0/text_b5_without_inputs.txt"
  fn_completions: "openai_completions"
  completions_kwargs:
    model_name: "text-davinci-003"
    max_tokens: 200
    temperature: 1.0
  completion_parser_kwargs:
    outputs_to_match:
      1: '\n\(a\)'
      2: '\n\(b\)'
  batch_size: 5


For more information, please refer to [AlpacaEval](https://github.com/tatsu-lab/alpaca_eval/tree/main). In our code, we directly use AlpacaEval, with a minor modification to separate instructions with and without inputs.